# Interactive Map of pollution in Occitanie

The goal of this notebook is to provide an interactive map (using `folium`) comparing the level of ozone pollution in Occitanie and in Paris (Paris 13), or only in Occitanie. It will allow you to choose a month and visualize a map with colored circle showing the level of pollution in each station. There could be more than one station per city. In the different parts of these notebook, you will find studies at different time scales.

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.colors as colors

from numpy import array
from numpy import max
import numpy as np
import pandas as pd
import math
import folium
from download import download

# 1 - Monthly study in 2018:

To cover an entire year, only 2018 is available to study precisely the ozone level in Occitanie and Paris at the same time.

In [2]:
# We choose monthly data
url = "https://opendata.arcgis.com/datasets/3acfa2aa5c0346a18ba7749c6885e503_0.csv"
path_target = "datasets/Mesure_mensuelle_Region_Occitanie_Polluants_Principaux.csv"
download(url, path_target, replace=True)

paris_df = pd.read_csv('PA13_2018.csv', sep=';',
                          comment='#',
                          na_values="n/d",
                          converters={'heure': str})

file_sizes: 311kB [00:00, 854kB/s]                                              


Successfully downloaded file to datasets/Mesure_mensuelle_Region_Occitanie_Polluants_Principaux.csv


### Data treatment:

Treatment of Occitanie data: We select ozone and variables we care about, we also transform with a good format the date.

In [3]:
occ_df = pd.read_csv(path_target)
occ_df = occ_df[occ_df['nom_poll'] == 'O3'] # only ozone
occ_df['month'] = pd.to_datetime(occ_df['date_debut']).dt.to_period('M') # good format for month
variables = ['X', 'Y', 'nom_com', 'nom_station', 'valeur', 'month'] # variables we care about
occ_df = occ_df[variables]

C:\Users\megan\Anaconda3\lib\site-packages\pandas\core\arrays\datetimes.py:1269: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  UserWarning,


Treatment of Paris data: We reconstruct the data to have the same variables than Occitanie 

In [4]:
paris_df = paris_df[paris_df.date.isna()==False] # delete NaN row
paris_df = paris_df[paris_df['O3']!='n/d'] # no line without O3 data
paris_df['O3'] = paris_df['O3'].astype('float') # convert type data as float
paris_df['month'] = pd.to_datetime(paris_df['date']).dt.to_period('M') # good format for month
par2018 = paris_df.groupby('month').agg({'O3':'mean'}) # We only care about month in this study

par2018['month'] = pd.PeriodIndex(['2018-01', '2018-02', '2018-03', '2018-04', 
                    '2018-05', '2018-06', '2018-07', '2018-08', 
                    '2018-09', '2018-10','2018-11','2018-12'], dtype='period[M]', freq='M')



par2018['nom_com'] = ['PARIS']*12
par2018['nom_station'] = ['Paris 13ème']*12
par2018['X'] = [2.3488]*12
par2018['Y'] = [48.8534]*12
par2018['valeur'] = par2018['O3']
par2018 = par2018[variables]

In [5]:
par2018

,X,Y,nom_com,nom_station,valeur,month
month,,,,,,
2018-01,2.3488,48.8534,PARIS,Paris 13ème,41.920613,2018-01
2018-02,2.3488,48.8534,PARIS,Paris 13ème,39.754491,2018-02
2018-03,2.3488,48.8534,PARIS,Paris 13ème,49.143243,2018-03
2018-04,2.3488,48.8534,PARIS,Paris 13ème,55.182749,2018-04
2018-05,2.3488,48.8534,PARIS,Paris 13ème,56.391181,2018-05
2018-06,2.3488,48.8534,PARIS,Paris 13ème,60.399687,2018-06
2018-07,2.3488,48.8534,PARIS,Paris 13ème,71.207367,2018-07
2018-08,2.3488,48.8534,PARIS,Paris 13ème,50.294737,2018-08
2018-09,2.3488,48.8534,PARIS,Paris 13ème,39.766667,2018-09


In [6]:
df_2018 = pd.concat([occ_df, par2018]) # data frame with Paris and Occitanie data

In [7]:
# Standardized data for a good color scale
df_2018['standard'] = (df_2018[['valeur']] - np.mean(df_2018[['valeur']]))/ np.std(df_2018[['valeur']])

The available cities in this exemple are: 

In [8]:
df_2018.nom_com.unique()

array(['MILLAU', 'NIMES', 'BESSIERES', 'PEYRUSSE-VIEILLE', 'SAZE',
       'TOULOUSE', 'MONTGISCARD', 'BIARS-SUR-CERE', 'SAINT-ESTEVE',
       'BELESTA-EN-LAURAGAIS', 'CORNEILHAN', 'AGDE', 'LATTES', 'TARBES',
       'LA CALMETTE', 'SAINT-GELY-DU-FESC', 'MIRAMONT-DE-COMMINGES',
       'MONTPELLIER', 'LOURDES', 'PERPIGNAN', 'FRAISSE-SUR-AGOUT',
       'RODEZ', 'CARCASSONNE', 'MENDE', 'ALBI', 'CASTRES', 'PAMIERS',
       'VALLABREGUES', 'PARIS'], dtype=object)

In [9]:
df_2018.reset_index(drop = True)

,X,Y,nom_com,nom_station,valeur,month,standard
0,3.07218,44.1062,MILLAU,Millau Urbain,50.800000,2018-02,-0.605128
1,3.07218,44.1062,MILLAU,Millau Urbain,67.800000,2018-03,0.374121
2,3.07218,44.1062,MILLAU,Millau Urbain,77.700000,2018-04,0.944389
3,3.07218,44.1062,MILLAU,Millau Urbain,66.700000,2018-05,0.310758
4,3.07218,44.1062,MILLAU,Millau Urbain,66.900000,2018-06,0.322278
...,...,...,...,...,...,...,...
310,2.34880,48.8534,PARIS,Paris 13ème,50.294737,2018-08,-0.634232
311,2.34880,48.8534,PARIS,Paris 13ème,39.766667,2018-09,-1.240679
312,2.34880,48.8534,PARIS,Paris 13ème,36.401617,2018-10,-1.434516
313,2.34880,48.8534,PARIS,Paris 13ème,26.974895,2018-11,-1.977522


In [10]:
df_2018[df_2018.month == '2018-09']

,X,Y,nom_com,nom_station,valeur,month,standard
1175,3.072180,44.1062,MILLAU,Millau Urbain,70.200000,2018-09,0.512368
1192,1.595830,43.8039,BESSIERES,Bessières-ECONOTRE,61.100000,2018-09,-0.011818
1202,0.179722,43.6303,PEYRUSSE-VIEILLE,Peyrusse Vieille Rural,77.600000,2018-09,0.938629
1213,4.679690,43.9429,SAZE,Saze - rural,79.300000,2018-09,1.036554
1224,1.438610,43.6236,TOULOUSE,Toulouse-Mazades Urbain,65.800000,2018-09,0.258915
1233,1.571110,43.4575,MONTGISCARD,Montgiscard Urbain,72.200000,2018-09,0.627573
1246,2.839870,42.7198,SAINT-ESTEVE,Saint Estève - périurbain,75.400000,2018-09,0.811903
1257,1.821940,43.4414,BELESTA-EN-LAURAGAIS,Belesta en Lauragais Rural,77.600000,2018-09,0.938629
1267,3.188810,43.4068,CORNEILHAN,Biterrois-Narbonnais - Périurbain,70.000000,2018-09,0.500847
1278,3.504830,43.2878,AGDE,Agathois-Piscénois - Périurbain,79.600000,2018-09,1.053835


### Interactive map for different months in 2018:

In [15]:
import branca.colormap as cm

linear = cm.LinearColormap(
    ['green', 'yellow', 'red'],
    vmin=-3, vmax=1
)
# colors

In [16]:
from ipywidgets import interact  # widget manipulation
from IPython.display import HTML

def interactive_map(mois = '2018-02'):
    
    map_2018 = df_2018[df_2018['month'] == mois]
    
    map_int = folium.Map(location = [46, 2.15], 
                         zoom_start = 6, 
                         tiles = 'Stamen Terrain')
    
    for i in range(0, len(map_2018)):
        folium.Circle(
            location = [map_2018.iloc[i]['Y'], map_2018.iloc[i]['X']],
            popup = map_2018.iloc[i]['nom_station'],
            radius = map_2018.iloc[i]['valeur']*500,
            color = 'black',
            fill = True,
            fill_color = linear(map_2018.iloc[i]['standard']),
            fill_opacity = 0.5,
            opacity = 0.4,
        ).add_to(map_int)
    
    return(map_int)

In [17]:
interact(interactive_map, mois=df_2018.month.unique())

interactive(children=(Dropdown(description='mois', options=(Period('2018-02', 'M'), Period('2018-03', 'M'), Pe…

<function __main__.interactive_map(mois='2018-02')>

## 2  - Another study
